In [145]:
import cv2
import numpy as np

In [146]:
from pathlib import Path
from typing import ClassVar, Tuple, Final
from os.path import exists

BACKGROUND_BRIGHTNESS: Final[int] = 0
FIRST_PLAN_BRIGHTNESS: Final[int] = 255


class FirstPlanObjectDetection:
	destination_directory: ClassVar[Path] = Path('./data')
	ground_truth_directory: ClassVar[Path] = Path('groundtruth')
	input_directory: ClassVar[Path] = Path('input')
	roi_directory: ClassVar[Path] = Path('temporalROI.txt')
	
	def __init__(self, data_file: Path) -> None:
		self.base_directory: Path = FirstPlanObjectDetection.destination_directory / Path(data_file.stem).stem
		if not exists(self.base_directory):
			import tarfile
			with tarfile.open(data_file, 'r:xz') as tar_file:
				tar_file.extractall(self.destination_directory)
		
		with open(self.base_directory / FirstPlanObjectDetection.roi_directory, 'r') as roi_file:
			self.start, self.end = map(int, roi_file.read().split())
		
		self.step: Final[int] = 1
		self.image_input: str = (self.base_directory / FirstPlanObjectDetection.input_directory / 'in%06d.jpg').absolute().as_posix()
		self.image_ground_truth_input: str = (self.base_directory / FirstPlanObjectDetection.ground_truth_directory / 'gt%06d.png').absolute().as_posix()
		
		print(self.base_directory.as_posix())
		
		# for index in range(self.start, self.end):
		# 	image: np.ndarray = cv2.imread(self.image_input % index)
		# 	
		# 	cv2.imshow('Image', image)
		# 	cv2.waitKey(10)
		# 	
		cv2.destroyAllWindows()
	
	def compile(self) -> Tuple[float, float, float]:
		second_image: np.ndarray = cv2.cvtColor(cv2.imread(self.image_input % self.start), cv2.COLOR_BGR2GRAY)

		true_positive: int = 0
		true_negative: int = 0
		false_positive: int = 0
		false_negative: int = 0
		
		for index in range(self.start, self.end, self.step):
			first_image: np.ndarray = second_image
			second_image: np.ndarray = cv2.cvtColor(
				cv2.imread(self.image_input % (index + self.step)),
				cv2.COLOR_BGR2GRAY
			)
			ground_truth: np.ndarray = cv2.imread(
                self.image_ground_truth_input % index,
				cv2.IMREAD_GRAYSCALE
			)
			
			ground_truth: np.ndarray = cv2.threshold(ground_truth, 165, 255, cv2.THRESH_BINARY)[1]
		
			difference: np.ndarray = cv2.absdiff(first_image, second_image.copy())
			binary: np.ndarray = cv2.threshold(difference, 13, 255, cv2.THRESH_BINARY)[1]
			median: np.ndarray = cv2.medianBlur(binary, 5)
			dilated: np.ndarray = cv2.dilate(median, np.ones((3, 3)), iterations=2)
			eroded: np.ndarray = cv2.erode(dilated, np.ones((5, 5)), iterations=1)
			dilated: np.ndarray = cv2.dilate(eroded, np.ones((3, 3)), iterations=1)
			eroded: np.ndarray = cv2.erode(dilated, np.ones((3, 3)), iterations=1)
			dilated: np.ndarray = cv2.dilate(eroded, np.ones((3, 3)), iterations=1)
			eroded: np.ndarray = cv2.erode(dilated, np.ones((3, 3)), iterations=1)
			dilated: np.ndarray = cv2.dilate(eroded, np.ones((5, 5)), iterations=2)
			eroded: np.ndarray = cv2.erode(dilated, np.ones((3, 3)), iterations=1)
		
			retval, labels, stats, centroids = cv2.connectedComponentsWithStats(eroded)
		
			image_visual: np.ndarray = eroded.copy()
		
			if stats.shape[0] > 1: # are  there  any  objects
				tab = stats[1:, 4] # 4 columns  without  first  element
				pi = np.argmax(tab) # finding  the  index  of the  largest  item
				pi += 1 # increment  because  we want  the  index  in stats , not in tab
				# drawing a bbox
				cv2.rectangle(image_visual, (stats[pi, 0], stats[pi, 1]), (stats[pi, 0] + stats[pi, 2], stats[pi, 1] + stats[pi, 3]), (255, 0, 0), 2) # print  information  about  the  field  and the  number  of the  largest  element
				cv2.putText(image_visual, str(stats[pi, 4]), (stats[pi, 0], stats[pi, 1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0))
				cv2.putText(image_visual, str(pi), (np.uint8(centroids[pi, 0]), np.uint8(centroids[pi, 1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))
		
			true_positive_matrix: np.ndarray = np.logical_and((eroded == FIRST_PLAN_BRIGHTNESS), (ground_truth == FIRST_PLAN_BRIGHTNESS))
			true_negative_matrix: np.ndarray = np.logical_and((eroded == BACKGROUND_BRIGHTNESS), (ground_truth == BACKGROUND_BRIGHTNESS))
			false_positive_matrix: np.ndarray = np.logical_and((eroded == FIRST_PLAN_BRIGHTNESS), (ground_truth == BACKGROUND_BRIGHTNESS))
			false_negative_matrix: np.ndarray = np.logical_and((eroded == BACKGROUND_BRIGHTNESS), (ground_truth == FIRST_PLAN_BRIGHTNESS))
		
			true_positive += np.sum(true_positive_matrix)
			true_negative += np.sum(true_negative_matrix)
			false_positive += np.sum(false_positive_matrix)
			false_negative += np.sum(false_negative_matrix)
		
			cv2.imshow('Image', image_visual)
			cv2.waitKey(10)
		
		cv2.destroyAllWindows()
		precision: Final[float] = true_positive / (true_positive + false_positive)
		recall: Final[float] = true_positive / (true_positive + false_negative)
		f1_score: Final[float] = 2 * precision * recall / (precision + recall)
		return precision, recall, f1_score

In [147]:
print('Precision: %f\nRecall: %f\nF1 Score: %f' % FirstPlanObjectDetection(Path('./pedestrian.tar.xz')).compile())

data/pedestrian
Precision: 0.610297
Recall: 0.840603
F1 Score: 0.707171


In [148]:
print('Precision: %f\nRecall: %f\nF1 Score: %f' % FirstPlanObjectDetection(Path('./highway.tar.xz')).compile())

data/highway
Precision: 0.783233
Recall: 0.929259
F1 Score: 0.850020


In [149]:
print('Precision: %f\nRecall: %f\nF1 Score: %f' % FirstPlanObjectDetection(Path('./office.tar.xz')).compile())

data/office
Precision: 0.813360
Recall: 0.177864
F1 Score: 0.291896
